# 1 - Consturcción de Dat Set Inicial 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

# ---------------------------------------------------
# 1. Carga y Preprocesamiento Inicial
# ---------------------------------------------------
print("1. Cargando y preparando datos...")

# Cargar datos
df = pd.read_csv('sell-in.txt', sep='\s+')
#df_sell_in = pd.read_csv('sell-in.txt', sep='\s+')
df_stocks = pd.read_csv('tb_stocks.txt', sep='\s+')
df_productos = pd.read_csv('tb_productos.txt', sep='\t')
df_target_products = pd.read_csv('product_id_apredecir201912.txt', sep='\t')


# Convertir periodo a datetime y ordenar
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')
df = df.sort_values(['product_id', 'periodo'])

# Agrupar por producto y periodo, sumando las tn
df_agrupado = df.groupby(['product_id', 'periodo'])['tn'].sum().reset_index()

# Opcional: Ordenar por producto y fecha
df = df_agrupado.sort_values(['product_id', 'periodo'])

# tn_0 es el tn actual
#df.rename(columns={"tn": "tn_0"}, inplace=True)

# Clase: tn del mes +2
df["tn+2"] = df.groupby("product_id")["tn"].shift(-2)

# ---------------------------------------------------
# 2. Feature Engineering Básico 
# ---------------------------------------------------

# Generar los lags tn_1 a tn_11 y clase = tn en periodo+2
for i in range(1, 12):
    df[f"tn_{i}"] = df.groupby("product_id")["tn"].shift(i)
    
    
print("2. Generando features básicos...")

def generate_features(group):
    group = group.sort_values('periodo')
    target_col = 'tn'
    
    # 1. Estadísticas móviles (versión compatible)
    windows = [3, 6, 12]
    for w in windows:
        group[f'{target_col}_avg_{w}'] = group[target_col].rolling(window=w, min_periods=1).mean()
        group[f'{target_col}_std_{w}'] = group[target_col].rolling(window=w, min_periods=1).std()
        group[f'{target_col}_min_{w}'] = group[target_col].rolling(window=w, min_periods=1).min()
        group[f'{target_col}_max_{w}'] = group[target_col].rolling(window=w, min_periods=1).max()
        group[f'{target_col}_median_{w}'] = group[target_col].rolling(window=w, min_periods=1).median()
    
    # 2. Diferencias y cambios porcentuales
    lags = [1, 3, 12]
    for lag in lags:
        group[f'{target_col}_diff_{lag}'] = group[target_col].diff(periods=lag)
        group[f'{target_col}_pct_{lag}'] = group[target_col].pct_change(periods=lag)
    
    # 3. Características de tendencia (versión segura)
    def calculate_trend(x):
        if len(x) < 2:
            return np.nan
        return np.polyfit(range(len(x)), x, 1)[0]
    
    group[f'{target_col}_trend_3'] = group[target_col].rolling(window=3).apply(calculate_trend, raw=True)
    
    # 4. Autocorrelación manual (versión compatible)
    def calculate_autocorr(x, lag):
        if len(x) < lag + 1:
            return np.nan
        return pd.Series(x).autocorr(lag=lag)
    
    for lag in [1, 3, 6]:
        group[f'autocorr_{lag}'] = group[target_col].rolling(window=lag+10).apply(
            lambda x: calculate_autocorr(x, lag), raw=True
        )
    
    # 5. Características de clientes (versión corregida)
    if 'customer_id' in group.columns:
        # Alternativa a rolling().nunique() que funciona en Pandas 2.1.4
        group['clientes_unicos_3'] = group['customer_id'].rolling(window=3, min_periods=1).apply(
            lambda x: pd.Series(x).nunique(), raw=False
        )
    
    return group

# Aplicar a cada producto (con manejo de errores)
try:
    df_features = df.groupby('product_id', group_keys=False).apply(generate_features)
except Exception as e:
    print(f"Error durante el feature engineering: {str(e)}")
    # Versión alternativa más robusta
    df_features = pd.concat([
        generate_features(group) for _, group in df.groupby('product_id')
    ])

# ---------------------------------------------------
# 3. Feature Engineering Avanzado Alternativo
# ---------------------------------------------------
print("3. Generando features avanzados alternativos...")

def advanced_features(group):
    group = group.sort_values('periodo')
    target = 'tn'
    
    # 1. Transformada de Fourier (versión segura)
    def get_fft(x):
        if len(x) < 3:
            return [np.nan, np.nan]
        try:
            fft = np.fft.fft(x)
            return [np.abs(fft[1]), np.angle(fft[1])]
        except:
            return [np.nan, np.nan]
    
    fft_results = group[target].rolling(window=12, min_periods=3).apply(
        lambda x: get_fft(x)[0], raw=True
    )
    group['fft_abs'] = fft_results
    
    # 2. Entropía aproximada (versión simplificada)
    group['entropy_6'] = group[target].rolling(window=6).apply(
        lambda x: np.log(np.std(x) + 1e-6) if len(x) > 1 else np.nan, raw=True
    )
    
    # 3. Cambios abruptos (versión robusta)
    group['change_points'] = group[target].rolling(window=6).apply(
        lambda x: np.sum(np.abs(np.diff(x)) > 2*np.std(x)) if len(x) > 1 else 0, raw=True
    )
    
    return group

# Aplicar con manejo de errores
try:
    df_features = df_features.groupby('product_id', group_keys=False).apply(advanced_features)
except Exception as e:
    print(f"Error en features avanzados: {str(e)}")
    df_features = pd.concat([
        advanced_features(group) for _, group in df_features.groupby('product_id')
    ])


<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:15: SyntaxWarning: invalid escape sequence '\s'
<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:15: SyntaxWarning: invalid escape sequence '\s'
C:\Users\01994\AppData\Local\Temp\ipykernel_26444\1714545469.py:13: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv('sell-in.txt', sep='\s+')
C:\Users\01994\AppData\Local\Temp\ipykernel_26444\1714545469.py:15: SyntaxWarning: invalid escape sequence '\s'
  df_stocks = pd.read_csv('tb_stocks.txt', sep='\s+')


1. Cargando y preparando datos...
2. Generando features básicos...
3. Generando features avanzados alternativos...


In [2]:
df_features

,product_id,periodo,tn,tn+2,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,...,tn_pct_3,tn_diff_12,tn_pct_12,tn_trend_3,autocorr_1,autocorr_3,autocorr_6,fft_abs,entropy_6,change_points
0,20001,2017-01-01,934.77222,1303.35771,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,2017-02-01,798.01620,1069.96130,934.77222,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20001,2017-03-01,1303.35771,1502.20132,798.01620,934.77222,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,184.292745,NaN,NaN,NaN,452.729232,NaN,NaN
3,20001,2017-04-01,1069.96130,1520.06539,1303.35771,798.01620,934.77222,NaN,NaN,NaN,...,0.144622,NaN,NaN,135.972550,NaN,NaN,NaN,458.049562,NaN,NaN
4,20001,2017-05-01,1502.20132,1030.67391,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,...,0.882420,NaN,NaN,99.421805,NaN,NaN,NaN,599.106693,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31238,21295,2017-01-01,0.00699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31239,21296,2017-08-01,0.00651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31240,21297,2017-01-01,0.00579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31241,21298,2017-08-01,0.00573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# ---------------------------------------------------
# 4. Manejo de Valores Faltantes (Versión Autónoma)
# ---------------------------------------------------
print("4. Limpiando datos faltantes...")

def clean_missing_values(df):
    """
    Función autónoma para manejo de valores faltantes que:
    1. Elimina columnas completamente vacías
    2. Imputa valores según el tipo de feature
    3. Elimina columnas con más del 30% de valores faltantes
    """
    
    # 1. Eliminar columnas completamente vacías
    empty_cols = [col for col in df.columns if df[col].isna().all()]
    if empty_cols:
        print(f"Eliminando columnas vacías: {empty_cols}")
        df = df.drop(columns=empty_cols)
    
    # 2. Estrategias de imputación por tipo de feature
    for col in df.columns:
        if col == 'product_id':
            continue  # No aplicar a la columna de identificación
            
        # Para cada grupo de producto
        for product_id, group in df.groupby('product_id'):
            mask = df['product_id'] == product_id
            col_data = df.loc[mask, col]
            
            # a) Features estadísticos (promedios, std, etc.)
            if any(x in col for x in ['_avg_', '_std_', '_min_', '_max_', '_median_']):
                # Imputación con media expansiva
                expanding_mean = col_data.expanding().mean()
                df.loc[mask, col] = col_data.fillna(expanding_mean)
            
            # b) Features avanzados (tendencias, autocorrelación)
            elif any(x in col for x in ['_trend_', 'autocorr_', 'fft_', 'entropy_']):
                # Imputación con mediana del grupo
                median_val = col_data.median()
                df.loc[mask, col] = col_data.fillna(median_val)
            
            # c) Otros features
            else:
                # Forward fill + relleno con cero
                filled = col_data.fillna(method='ffill').fillna(0)
                df.loc[mask, col] = filled
    
    # 3. Eliminar columnas con muchos valores faltantes
    threshold = int(0.7 * len(df))  # 70% de los datos como umbral
    cols_to_drop = [col for col in df.columns if df[col].isna().sum() > (len(df) - threshold)]
    
    if cols_to_drop:
        print(f"Eliminando columnas con >30% NAs: {cols_to_drop}")
        df = df.drop(columns=cols_to_drop)
    
    return df

# Aplicar la función
try:
    df_features = clean_missing_values(df_features)
    print("Limpieza de valores faltantes completada exitosamente!")
except Exception as e:
    print(f"Error durante la limpieza: {str(e)}")
    # Opción de respaldo básica
    df_features = df_features.fillna(0).dropna(axis=1, how='all')

4. Limpiando datos faltantes...
Limpieza de valores faltantes completada exitosamente!


In [21]:
df_features.head()

,product_id,periodo,tn,tn+2,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,...,tn_pct_3,tn_diff_12,tn_pct_12,tn_trend_3,autocorr_1,autocorr_3,autocorr_6,fft_abs,entropy_6,change_points
0,20001,2017-01-01,934.77222,1303.35771,0.00000,0.00000,0.00000,0.00000,0.0,0.0,...,0.000000,0.0,0.0,77.633350,-0.070356,-0.342902,-0.086148,841.961582,5.385568,0.0
1,20001,2017-02-01,798.01620,1069.96130,934.77222,0.00000,0.00000,0.00000,0.0,0.0,...,0.000000,0.0,0.0,77.633350,-0.070356,-0.342902,-0.086148,841.961582,5.385568,0.0
2,20001,2017-03-01,1303.35771,1502.20132,798.01620,934.77222,0.00000,0.00000,0.0,0.0,...,0.000000,0.0,0.0,184.292745,-0.070356,-0.342902,-0.086148,452.729232,5.385568,0.0
3,20001,2017-04-01,1069.96130,1520.06539,1303.35771,798.01620,934.77222,0.00000,0.0,0.0,...,0.144622,0.0,0.0,135.972550,-0.070356,-0.342902,-0.086148,458.049562,5.385568,0.0
4,20001,2017-05-01,1502.20132,1030.67391,1069.96130,1303.35771,798.01620,934.77222,0.0,0.0,...,0.882420,0.0,0.0,99.421805,-0.070356,-0.342902,-0.086148,599.106693,5.385568,0.0


In [18]:
# ---------------------------------------------------
# 5. Exportar Resultados
# ---------------------------------------------------
def safe_export_to_csv(df, filename):
    """Exporta un DataFrame a CSV usando Python puro"""
    try:
        # Obtener columnas
        columns = df.columns.tolist()
        
        # Abrir archivo en modo escritura
        with open(filename, 'w', encoding='utf-8') as f:
            # Escribir encabezados
            f.write(','.join(columns) + '\n')
            
            # Escribir filas
            for _, row in df.iterrows():
                line = ','.join([str(row[col]) for col in columns]) + '\n'
                f.write(line)
                
        print(f"Archivo guardado exitosamente en {filename}")
        return True
    except Exception as e:
        print(f"Error al exportar: {str(e)}")
        return False

# Uso:
safe_export_to_csv(df_features, 'sell_in_features_final.csv')

Archivo guardado exitosamente en sell_in_features_final.csv


True